In [1]:
import numpy as np
import skimage as sk
from aicspylibczi import CziFile
from glob import glob
import os
import napari

In [ ]:
OIC_191_files = sorted(glob('OIC-191_Images/*.czi'))
OIC_192_files = sorted(glob('OIC-192_Images/*.czi'))

OIC_191_files_czi = list(map(CziFile,OIC_191_files))
OIC_192_files_czi = list(map(CziFile,OIC_192_files))

In [4]:
new_images_files = sorted(glob('20250905_Images/*.czi'))
new_images_czi = list(map(CziFile,new_images_files))

In [7]:
len(new_images_files)

8

In [5]:
def read_and_prep_images(czifile):
    num_scenes = czifile.size[2]
    scenes = [czifile.read_image(S=s)[0] for s in range(czifile.size[2])]
    squeezed = [np.squeeze(i) for i in scenes]
    num_channels = squeezed[0].shape[0]
    average_projections = [np.round(np.mean(img,axis=1)).astype('uint16') for img in squeezed] #channel axis needs to be placed last for scikit-image
    montage = np.block(average_projections)
    return num_scenes, num_channels, average_projections, montage 

In [6]:
def save_imgs(save_path,file_name, num_scenes, num_channels, average_projections, montage):
    for i in range(num_scenes):
        scene = average_projections[i]
        sk.io.imsave(os.path.join(save_path,file_name[:-4]+'_AverageProjection_Scene0'+str(i+1)+'.tiff'),scene,check_contrast=False)
        for c in range(num_channels):
            ch = scene[c,...]
            sk.io.imsave(os.path.join(save_path,file_name[:-4]+'_AverageProjection_Scene0'+str(i+1)+'_Channel'+str(c+1)+'.tiff'),ch,check_contrast=False)
    sk.io.imsave(os.path.join(save_path,file_name[:-4]+'_AverageProjection_Montage.tiff'),montage,check_contrast=False)


In [8]:
save_path = '20250905_Images/Average_Projections_Montages'
for j in range(len(new_images_czi)):
    file_name = os.path.basename(new_images_files[j])
    img = new_images_czi[j]
    num_scenes, num_channels, average_projections, montage = read_and_prep_images(img)
    save_imgs(save_path,file_name,num_scenes,num_channels,average_projections,montage)